In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

spark = SparkSession.builder \
    .appName("Most Popular Destinations with MapReduce") \
    .getOrCreate()

df = spark.read.parquet("hdfs://namenode:9000/data/data_trans/")

df = df.select("origin_id", "destination_id", "price_cents")

df = df.na.drop(subset=["price_cents", "destination_id"])

rdd = df.rdd.map(lambda row: (row["destination_id"], (1, row["price_cents"])))

reduced_rdd = rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

average_price_rdd = reduced_rdd.map(lambda x: (x[0], x[1][0], x[1][1] / x[1][0]))

popular_destinations_df = average_price_rdd.toDF(["destination_id", "trip_count", "avg_price_cents"])

popular_destinations_df = popular_destinations_df.sort(desc("trip_count"))

popular_destinations_df.show(10)

